In [10]:
import numpy as np
import pandas as pd



In [11]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']

df = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item', sep='|', names=columns, encoding='latin-1')

movie_names = movies[['item_id', 'movie title']]
combined_movies_data = pd.merge(df, movie_names, on='item_id')
combined_movies_data = combined_movies_data[['user_id','movie title', 'rating']]
combined_movies_data.head()

,user_id,movie title,rating
0,196,Kolya (1996),3
1,63,Kolya (1996),3
2,226,Kolya (1996),5
3,154,Kolya (1996),3
4,306,Kolya (1996),5


In [16]:
my_ratings = pd.read_csv('myratings.csv')
my_ratings

,User ID,Movie,Ratings
0,123,Star Wars A New Hope (1977),4.5
1,123,Braveheart (1995),4.0
2,123,The Lion King (2019),3.8
3,123,Titanic (1997),3.9
4,123,The Matrix (1999),4.4
5,123,Wall-E (2008),4.5
6,123,Deadpool (2016),4.3
7,123,The Terminator (1984),3.4
8,123,The Devil's Violinist (2013),2.0
9,123,The Devil's Wears Prada (2006),1.9


In [18]:
combined_movies_data = pd.concat([combined_movies_data, my_ratings], axis=0)

# rename the columns to userID, itemID and rating
combined_movies_data.columns = ['userID', 'itemID', 'rating', 'movieTitle', 'year', 'title']

# use the transform method group by userID and count to keep the movies with more than 25 reviews
combined_movies_data['reviews'] = combined_movies_data.groupby(['itemID'])['rating'].transform('count')

combined_movies_data= combined_movies_data[combined_movies_data.reviews>25][['userID', 'itemID', 'rating']]

In [20]:
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset

In [21]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_movies_data, reader)

In [22]:
# get the list of the movie ids
unique_ids = combined_movies_data['itemID'].unique()

# get the list of the ids that the userid 1001 has rated
iids1001 = combined_movies_data.loc[combined_movies_data['userID']==1001, 'itemID']

# remove the rated movies for the recommendations
movies_to_predict = np.setdiff1d(unique_ids,iids1001)

In [23]:
algo = SVD()
algo.fit(data.build_full_trainset())
my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=1001,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
171,"Close Shave, A (1995)",4.547188
657,Schindler's List (1993),4.533540
845,"Wrong Trousers, The (1993)",4.523851
143,Casablanca (1942),4.512100
679,"Shawshank Redemption, The (1994)",4.494483
821,Wallace & Gromit: The Best of Aardman Animatio...,4.432816
621,Rear Window (1954),4.423631
1,12 Angry Men (1957),4.409112
809,"Usual Suspects, The (1995)",4.392920
763,"Third Man, The (1949)",4.389623
